# Clase 7: Taller Práctico - Otros Métodos Supervisados

**Objetivos:**

En este taller, aplicaremos y compararemos los tres métodos de clasificación que hemos estudiado en la clase teórica:

1.  **k-Nearest Neighbors (k-NN)**: Un clasificador basado en instancia y distancia.
2.  **Naive Bayes**: Un clasificador probabilístico basado en el teorema de Bayes.
3.  **Perceptrón Multicapa (MLP)**: Nuestro primer vistazo a una red neuronal simple.

Utilizaremos el conjunto de datos "Wine" de Scikit-learn, un dataset clásico para problemas de clasificación multiclase. El objetivo es clasificar vinos en una de tres categorías basándonos en sus atributos químicos.

## 1. Configuración Inicial e Importación de Librerías

In [1]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

import time

# Librerías de Scikit-learn para el modelado
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA

# Librerías para visualización
import plotly.express as px
import plotly.graph_objects as go

## 2. Carga y Exploración del Dataset (EDA)

El dataset "Wine" contiene los resultados de un análisis químico de vinos cultivados en la misma región en Italia pero derivados de tres cultivares diferentes. El análisis determinó las cantidades de 13 constituyentes encontrados en cada uno de los tres tipos de vinos.

In [2]:
# Cargar el dataset
wine_data = load_wine()
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
y = pd.Series(wine_data.target, name='target')

# Unir características y objetivo en un solo DataFrame para exploración
df = pd.concat([X, y], axis=1)

# Mapear los números del objetivo a nombres de clases para mayor claridad
df['target_name'] = df['target'].map({0: 'Class_0', 1: 'Class_1', 2: 'Class_2'})

print("Primeras 5 filas del dataset:")
display(df.head())

print("\nDescripción estadística de las características:")
display(df.describe())

Primeras 5 filas del dataset:


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target,target_name
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0,Class_0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0,Class_0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0,Class_0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0,Class_0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0,Class_0



Descripción estadística de las características:


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


### Visualización con PCA

Dado que tenemos 13 características, no podemos visualizarlas todas a la vez. Usaremos el Análisis de Componentes Principales (PCA) para reducir la dimensionalidad a 2 componentes y visualizar la separación de las clases.

In [3]:
# Primero, escalamos los datos para que PCA funcione correctamente
scaler_pca = StandardScaler()
X_scaled_pca = scaler_pca.fit_transform(X)

# Aplicamos PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_pca)

# Creamos un DataFrame para la visualización
df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
df_pca['target_name'] = df['target_name']

# Gráfico interactivo con Plotly
fig = px.scatter(df_pca, 
                 x='PC1', 
                 y='PC2', 
                 color='target_name', 
                 title='Visualización del Dataset Wine con PCA (2 Componentes)',
                 labels={'PC1': 'Primer Componente Principal', 'PC2': 'Segundo Componente Principal'},
                 template='plotly_white')

fig.show()

La visualización PCA nos muestra que las clases están razonablemente bien separadas, aunque con algo de superposición. Esto sugiere que los algoritmos de clasificación deberían poder encontrar patrones útiles.

## 3. Preparación de los Datos para el Modelado

Ahora, dividiremos los datos en conjuntos de entrenamiento y prueba, y aplicaremos el escalado de características. El escalado es fundamental para k-NN y MLP, ya que son sensibles a las diferentes escalas de las variables.

In [4]:
# Separar los datos originales (sin PCA) en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar el escalador SÓLO con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la misma transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test)

print(f"Tamaño del set de entrenamiento: {X_train_scaled.shape[0]} muestras")
print(f"Tamaño del set de prueba: {X_test_scaled.shape[0]} muestras")

Tamaño del set de entrenamiento: 124 muestras
Tamaño del set de prueba: 54 muestras


## 4. Implementación y Evaluación de Modelos

### Modelo 1: k-Nearest Neighbors (k-NN)

Implementaremos k-NN con un valor de `k` inicial (ej. k=5) y evaluaremos su rendimiento.

In [5]:
start = time.time()
# Inicializar y entrenar el clasificador k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_knn = knn.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de k-NN (k=5) ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")

print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_knn))

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo k-NN")

------ Resultados de k-NN (k=5) ------
Accuracy: 0.9444

Matriz de Confusión:
[[18  0  0]
 [ 0 18  3]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.86      0.92        21
           2       0.83      1.00      0.91        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54

0.018 segundos para entrenar el modelo k-NN


#### Desafío: Encontrar el `k` Óptimo

El rendimiento de k-NN depende fuertemente del valor de `k`. Un `k` muy pequeño puede llevar a sobreajuste, mientras que un `k` muy grande puede simplificar demasiado el modelo. 

**Tu tarea:** Escribe un bucle que entrene y evalúe el modelo k-NN para un rango de valores de `k` (por ejemplo, de 1 a 30). Almacena la precisión (accuracy) para cada `k` y luego crea un gráfico para visualizar cómo cambia la precisión en función de `k`. ¿Cuál es el valor de `k` que da el mejor resultado en el conjunto de prueba?

In [6]:
# Espacio para tu solución al desafío
k_values = range(1, 31)
accuracies = []

for k in k_values:
    knn_loop = KNeighborsClassifier(n_neighbors=k)
    knn_loop.fit(X_train_scaled, y_train)
    y_pred_loop = knn_loop.predict(X_test_scaled)
    accuracies.append(accuracy_score(y_test, y_pred_loop))

# Gráfico de k vs. Accuracy
fig = go.Figure(data=go.Scatter(x=list(k_values), y=accuracies, mode='lines+markers'))
fig.update_layout(
    title='Accuracy de k-NN en función del número de vecinos (k)',
    xaxis_title='Valor de k',
    yaxis_title='Accuracy en el conjunto de prueba',
    template='plotly_white'
)
fig.show()

best_k = k_values[np.argmax(accuracies)]
print(f"\nEl valor óptimo de k es: {best_k} con una precisión de {max(accuracies):.4f}")


El valor óptimo de k es: 21 con una precisión de 1.0000


### Modelo 2: Naive Bayes

Ahora, aplicaremos el clasificador Naive Bayes Gaussiano. Esta variante es adecuada porque nuestras características son continuas y podemos asumir (ingenuamente) que siguen una distribución normal dentro de cada clase.

In [7]:
start = time.time()
# Inicializar y entrenar el clasificador Naive Bayes Gaussiano
# Nota: Naive Bayes no es tan sensible al escalado, pero es buena práctica usar los datos escalados por consistencia.
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_gnb = gnb.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de Naive Bayes Gaussiano ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gnb):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_gnb))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_gnb))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo Naive Bayes")

------ Resultados de Naive Bayes Gaussiano ------
Accuracy: 1.0000

Matriz de Confusión:
[[18  0  0]
 [ 0 21  0]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        15

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

0.012 segundos para entrenar el modelo Naive Bayes


### Modelo 3: Perceptrón Multicapa (MLP)

Finalmente, implementaremos una red neuronal simple. Usaremos `MLPClassifier` de Scikit-learn, que es una implementación eficiente y fácil de usar. Definiremos una arquitectura simple con una capa oculta.

In [8]:
start = time.time()
# Inicializar y entrenar el clasificador MLP
# hidden_layer_sizes=(100,) significa una capa oculta con 100 neuronas.
# max_iter=1000 para asegurar la convergencia.
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_mlp = mlp.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados del Perceptrón Multicapa (MLP) ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_mlp):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_mlp))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_mlp))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo MLP")

------ Resultados del Perceptrón Multicapa (MLP) ------
Accuracy: 1.0000

Matriz de Confusión:
[[18  0  0]
 [ 0 21  0]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        15

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

0.180 segundos para entrenar el modelo MLP


## 5. Comparación y Conclusiones

En este caso particular, los tres modelos han obtenido una precisión muy alta, a menudo perfecta o casi perfecta en el conjunto de prueba. Esto se debe a que el dataset Wine es un problema relativamente "fácil" con clases bien separadas.

* **k-NN** demostró ser muy efectivo, especialmente después de encontrar un buen valor para `k`.
* **Naive Bayes** también funcionó excepcionalmente bien, lo que sugiere que, para este problema, el supuesto de independencia condicional no fue una limitación grave.
* El **MLP** igualó el rendimiento de los otros modelos, mostrando su capacidad para resolver problemas de clasificación, aunque su entrenamiento es computacionalmente más costoso.

En problemas del mundo real más complejos y con mayor superposición entre clases, las diferencias en el rendimiento de estos algoritmos suelen ser más pronunciadas. La elección del modelo dependerá de las características específicas del problema, la cantidad de datos disponibles y los recursos computacionales.

## 6. Ejercicios Adicionales

Para solidificar tu comprensión, aquí tienes 10 ejercicios para explorar por tu cuenta.

1.  **Cambiar el Dataset:** Carga el dataset `load_breast_cancer` de `sklearn.datasets`. Es un problema de clasificación binaria. Repite el análisis completo (EDA, preprocesamiento, modelado y comparación) para este nuevo dataset. ¿Qué modelo funciona mejor aquí?

2. **Arquitectura del MLP:** Experimenta con diferentes arquitecturas para el `MLPClassifier`. Prueba con más capas ocultas (ej. `hidden_layer_sizes=(100, 50,)`) o con diferente número de neuronas por capa. ¿Puedes mejorar la precisión obtenida?

3. **Métricas de Distancia en k-NN:** El `KNeighborsClassifier` tiene un parámetro `metric`. Por defecto es `'minkowski'` (que con p=2 es la distancia Euclidiana). Prueba a cambiarlo a `'manhattan'` (Distancia de Manhattan). ¿Cambia el rendimiento del modelo?

4. **Robustez del `train_test_split`:** Cambia el valor de `random_state` en la función `train_test_split`. Repite el entrenamiento y evaluación de los tres modelos. ¿Son los resultados de precisión exactamente los mismos? ¿Qué nos dice esto sobre la evaluación de modelos?

5. **Comparar con Regresión Logística:** Como un modelo de base adicional, importa `LogisticRegression` de `sklearn.linear_model`. Entrénalo y evalúalo en el dataset Wine. ¿Cómo se compara su rendimiento con los tres modelos de esta clase?

In [ ]:
""" 
1)
"""
from sklearn.datasets import load_breast_cancer

# Cargar el dataset de cáncer de mama
data = load_breast_cancer(as_frame=True)
df = data.frame
X = df[data.feature_names]
y = df['target']

# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# k-NN
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("------ Resultados de k-NN (k=15) para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_knn))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))

# Naive Bayes Gaussiano
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)
y_pred_gnb = gnb.predict(X_test_scaled)

print("------ Resultados de Naive Bayes Gaussiano para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gnb):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_gnb))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_gnb))

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print("------ Resultados del Perceptrón Multicapa (MLP) para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_mlp):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_mlp))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_mlp))

""" 
El modelo que mejor funciona en el dataset de cáncer de mama es el Perceptrón Multicapa (MLP) con una precisión de 0.9825, seguido por k-NN con k=15 con una precisión de 0.9591 
y Naive Bayes Gaussiano con una precisión de 0.9357
"""

------ Resultados de k-NN (k=15) para el dataset de cáncer de mama ------
Accuracy: 0.9591

Matriz de Confusión:
[[ 57   7]
 [  0 107]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        64
           1       0.94      1.00      0.97       107

    accuracy                           0.96       171
   macro avg       0.97      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171

------ Resultados de Naive Bayes Gaussiano para el dataset de cáncer de mama ------
Accuracy: 0.9357

Matriz de Confusión:
[[ 57   7]
 [  4 103]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        64
           1       0.94      0.96      0.95       107

    accuracy                           0.94       171
   macro avg       0.94      0.93      0.93       171
weighted avg       0.94      0.94      0.94       171

------ Res

In [16]:
""" 
2)
"""
# MLP con una capa oculta de 100 neuronas, 1000 iteraciones y funcion de activacion relu (default)
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print(f"Accuracy hidden_layer_sizes=(100,) y relu: {accuracy_score(y_test, y_pred_mlp):.4f}")

# MLP con dos capas ocultas de 100 y 50 neuronas, 1000 iteraciones y funcion de activacion relu (default)
mlp_2_layers = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_2_layers.fit(X_train_scaled, y_train)
y_pred_mlp_2_layers = mlp_2_layers.predict(X_test_scaled)

print(f"Accuracy hidden_layer_sizes=(100, 50) y relu: {accuracy_score(y_test, y_pred_mlp):.4f}")

# MLP con una capa oculta de 100 neuronas, 1000 iteraciones y funcion de activacion sigmoide
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, activation='logistic')
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print(f"Accuracy hidden_layer_sizes=(100,) y sigmoide: {accuracy_score(y_test, y_pred_mlp):.4f}")

# MLP con dos capas ocultas de 100 y 50 neuronas, 1000 iteraciones y funcion de activacion sigmoide
mlp_2_layers = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42, activation='logistic')
mlp_2_layers.fit(X_train_scaled, y_train)
y_pred_mlp_2_layers = mlp_2_layers.predict(X_test_scaled)

print(f"Accuracy hidden_layer_sizes=(100, 50) y sigmoide: {accuracy_score(y_test, y_pred_mlp):.4f}")

# MLP con tres capas ocultas de 100, 50, 25 neuronas, 1000 iteraciones y funcion de activacion relu (default)
mlp = MLPClassifier(hidden_layer_sizes=(100,50,25), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print(f"Accuracy hidden_layer_sizes=(100,50,25) y relu: {accuracy_score(y_test, y_pred_mlp):.4f}")

# MLP con tres capas ocultas de 100, 50, 25 neuronas, 10000 iteraciones y funcion de activacion relu (default)
mlp_2_layers = MLPClassifier(hidden_layer_sizes=(100,50,25), max_iter=10000, random_state=42)
mlp_2_layers.fit(X_train_scaled, y_train)
y_pred_mlp_2_layers = mlp_2_layers.predict(X_test_scaled)

print(f"Accuracy hidden_layer_sizes=(100,50,25) y relu: {accuracy_score(y_test, y_pred_mlp):.4f}")

Accuracy hidden_layer_sizes=(100,) y relu: 0.9825
Accuracy hidden_layer_sizes=(100, 50) y relu: 0.9825
Accuracy hidden_layer_sizes=(100,) y sigmoide: 0.9708
Accuracy hidden_layer_sizes=(100, 50) y sigmoide: 0.9708
Accuracy hidden_layer_sizes=(100,50,25) y relu: 0.9766
Accuracy hidden_layer_sizes=(100,50,25) y relu: 0.9766


In [ ]:
""" 
3)
"""
knn = KNeighborsClassifier(n_neighbors=15, metric='manhattan')
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("------ Resultados de k-NN (k=15) y Manhattan para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_knn))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))

""" 
En este caso particular usando Manhattan para calcular la distancia se obtuvo una baja en la accuracy del modelo en comparacion con usar la distancia euclediana
"""

------ Resultados de k-NN (k=15) y Manhattan para el dataset de cáncer de mama ------
Accuracy: 0.9474

Matriz de Confusión:
[[ 56   8]
 [  1 106]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.98      0.88      0.93        64
           1       0.93      0.99      0.96       107

    accuracy                           0.95       171
   macro avg       0.96      0.93      0.94       171
weighted avg       0.95      0.95      0.95       171



In [ ]:
""" 
4)
"""
# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# k-NN
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("------ Resultados de k-NN (k=15) para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")

# Naive Bayes Gaussiano
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)
y_pred_gnb = gnb.predict(X_test_scaled)

print("------ Resultados de Naive Bayes Gaussiano para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gnb):.4f}")

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print("------ Resultados del Perceptrón Multicapa (MLP) para el dataset de cáncer de mama ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_mlp):.4f}")

""" 
Los resultados obtenidos son distintos, en todos los casos los resultados obtenidos son peores; cambiar el random state afecta la forma en la cual se dividen los datos
"""

------ Resultados de k-NN (k=15) para el dataset de cáncer de mama ------
Accuracy: 0.9415
------ Resultados de Naive Bayes Gaussiano para el dataset de cáncer de mama ------
Accuracy: 0.9123
------ Resultados del Perceptrón Multicapa (MLP) para el dataset de cáncer de mama ------
Accuracy: 0.9532


In [ ]:
""" 
5)
"""
from sklearn.linear_model import LogisticRegression

# Cargar el dataset
wine_data = load_wine()
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
y = pd.Series(wine_data.target, name='target')
df = pd.concat([X, y], axis=1)
df['target_name'] = df['target'].map({0: 'Class_0', 1: 'Class_1', 2: 'Class_2'})

# Escalamos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenamos el modelo con Regresion Logistica
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_scaled, y_train)
y_pred = log_reg.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de Regresion Logistica ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

""" 
El resultado obtenido es peor que el dado por los otros modelos 0.98 vs 1
"""

------ Resultados de Regresion Logistica ------
Accuracy: 0.9815

Matriz de Confusión:
[[18  0  0]
 [ 1 20  0]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        18
           1       1.00      0.95      0.98        21
           2       1.00      1.00      1.00        15

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54

